In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import mne
import numpy as np
import mat73
from mne_connectivity.viz import plot_connectivity_circle

import scipy.io
import numpy as np
import h5py as h5

import seaborn as sns

import mat73 

from scipy.signal import hilbert
import pandas as pd
from scipy.stats import zscore
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from fooof.plts.spectra import plot_spectrum
from fooof import FOOOF
from fooof.sim.gen import gen_aperiodic

import pickle # For reading dictionaries from .pkl

from scipy.stats import pearsonr

import networkx as nx

from scipy.stats import kruskal
from statsmodels.stats.multitest import fdrcorrection

In [2]:
cd

In [3]:
cd Documents/Projects/PROJECT_main/multiclass_meg_features_analysis/

In [4]:
from libs import neurofeat

In [5]:
# List of ROIs from the AAL atlas
ROI_AAL_list = [ 'Rectus_L','Olfactory_L','Frontal_Sup_Orb_L','Frontal_Med_Orb_L','Frontal_Mid_Orb_L',
                'Frontal_Inf_Orb_L','Frontal_Sup_L','Frontal_Mid_L','Frontal_Inf_Oper_L','Frontal_Inf_Tri_L',
                'Frontal_Sup_Medial_L','Supp_Motor_Area_L','Paracentral_Lobule_L','Precentral_L','Rolandic_Oper_L',
                'Postcentral_L','Parietal_Sup_L','Parietal_Inf_L','SupraMarginal_L','Angular_L','Precuneus_L',
                'Occipital_Sup_L','Occipital_Mid_L','Occipital_Inf_L','Calcarine_L','Cuneus_L','Lingual_L',
                'Fusiform_L','Heschl_L','Temporal_Sup_L','Temporal_Mid_L','Temporal_Inf_L','Temporal_Pole_Sup_L',
                'Temporal_Pole_Mid_L','ParaHippocampal_L','Cingulum_Ant_L','Cingulum_Mid_L','Cingulum_Post_L',
                'Insula_L','Rectus_R','Olfactory_R','Frontal_Sup_Orb_R','Frontal_Med_Orb_R','Frontal_Mid_Orb_R',
                'Frontal_Inf_Orb_R','Frontal_Sup_R','Frontal_Mid_R','Frontal_Inf_Oper_R','Frontal_Inf_Tri_R',
                'Frontal_Sup_Medial_R','Supp_Motor_Area_R','Paracentral_Lobule_R','Precentral_R','Rolandic_Oper_R',
                'Postcentral_R','Parietal_Sup_R','Parietal_Inf_R', 'SupraMarginal_R','Angular_R','Precuneus_R',
                'Occipital_Sup_R','Occipital_Mid_R','Occipital_Inf_R','Calcarine_R','Cuneus_R','Lingual_R',
                'Fusiform_R','Heschl_R','Temporal_Sup_R','Temporal_Mid_R','Temporal_Inf_R','Temporal_Pole_Sup_R',
                'Temporal_Pole_Mid_R','ParaHippocampal_R','Cingulum_Ant_R','Cingulum_Mid_R','Cingulum_Post_R',
                'Insula_R','Hippocampus_L','Hippocampus_R','Amygdala_L','Amygdala_R','Caudate_L','Caudate_R',
                'Putamen_L','Putamen_R','Pallidum_L','Pallidum_R','Thalamus_L','Thalamus_R','Cerebelum_Crus1_L',
                'Cerebelum_Crus1_R','Cerebelum_Crus2_L','Cerebelum_Crus2_R','Cerebelum_3_L','Cerebelum_3_R',
                'Cerebelum_4_5_L','Cerebelum_4_5_R','Cerebelum_6_L','Cerebelum_6_R','Cerebelum_7b_L','Cerebelum_7b_R',
                'Cerebelum_8_L','Cerebelum_8_R','Cerebelum_9_L','Cerebelum_9_R','Cerebelum_10_L','Cerebelum_10_R',
                'Vermis_1_2','Vermis_3','Vermis_4_5','Vermis_6','Vermis_7','Vermis_8','Vermis_9','Vermis_10']
ROIs = '%0d'.join ( ROI_AAL_list )

# For each ROI, it attributes one lobe
Region_AAL = [ 'LF','LF','LF','LF','LF','LF','LF','LF','LF','LF','LF','LF','LF','LF','LF','LP','LP','LP','LP','LP',
              'LP','LO','LO','LO','LO','LO','LO','LO','LT','LT','LT','LT','LT','LT','LT','LF','LP','LP','LT','RF',
              'RF','RF','RF','RF','RF','RF','RF','RF','RF','RF','RF','RF','RF','RF','RP','RP','RP','RP','RP','RP',
              'RO','RO','RO','RO','RO','RO','RO','RT','RT','RT','RT','RT','RT','RT','RF','RP','RP','RT','LT','RT',
              'NB','NB','NB','NB','NB','NB','NB','NB','NB','NB','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer',
              'Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer','Cer',
              'Cer','Cer' ] # partition provided by PS

In [6]:
from nilearn import datasets

aal = datasets.fetch_atlas_aal()

In [7]:
len(aal["labels"])

In [8]:
set(ROI_AAL_list) == set(aal["labels"])

In [9]:
indices_aal = [ROI_AAL_list.index(item) for item in aal["labels"]]

In [10]:
from nilearn import plotting

coordinates = plotting.find_parcellation_cut_coords(labels_img=aal["maps"])

In [11]:
len(coordinates) # coordinates of AAL atlas regions

In [12]:
# Load average feature importances estimated from XGB
avg_imp_aec_plv = pd.read_csv("data/average_importance_xgb/avg_imp_aec_plv.csv", index_col=0)
avg_imp_aec_plv_atm = pd.read_csv("data/average_importance_xgb/avg_imp_aec_plv_atm.csv", index_col=0)

ver_8apr_overlap = pd.read_csv("data/average_importance_xgb/overlap_feats_atm-aec-plv.csv", index_col=0)

In [13]:
avg_imp_aec_plv.shape, avg_imp_aec_plv_atm.shape, ver_8apr_overlap.shape

In [14]:
avg_imp_aec_plv_atm.index

In [15]:
ver_8apr_overlap.index

In [16]:
ver_8apr_overlap.index = [1182, 2563, 645, 2561, 5254, 2565, 5175, 4554, 1112]

In [17]:
# Find row and column indeces for the most important edges (2 regions)

matr_zer = np.zeros((116, 116))

for index, value in ver_8apr_overlap.iterrows():
    row, col = divmod(index, 100)
    print(f'row: {row}, id: {col}')
    matr_zer[row, col] = value.values[0]

In [18]:
matr_zer.shape

In [19]:
matr_zer.nonzero()

In [20]:
matr_zer

In [21]:
# Save a sparse matrix to .mat file 

from scipy.io import savemat

savemat(file_name="sparse_matrix.mat", mdict={"data": matr_zer})

In [22]:
# regions of AAL from library and our - are in different order so we should reorder them
matr_zer = matr_zer[indices_aal, :][:, indices_aal]

In [23]:
matr_zer.nonzero()

In [24]:
np.unique(np.array(np.nonzero(matr_zer)).flatten())

In [25]:
# find coordinates of specified regions
coordinates[[ 1,  2,  3, 18, 33, 43, 44, 47, 51, 57, 68, 69, 70], :]

In [26]:
symmetric_matrix = matr_zer + matr_zer.T # make a symmetric matrix to have undirected links

In [27]:
ver_8apr_overlap

In [28]:
np.where((symmetric_matrix > 0.027) & (symmetric_matrix < 0.0292))

In [29]:
symmetric_matrix[[18,70],:][:,[18,70]]

In [30]:
matr_zer.nonzero()

In [31]:
idx_nonzer = [1,  2,  3, 18, 33, 43, 44, 47, 51, 57, 68, 69, 70]
 
plotting.plot_connectome(
    #matr_zer[indices_aal, :][:, indices_aal],
    #symmetric_matrix,
    symmetric_matrix[idx_nonzer,:][:,idx_nonzer],
    coordinates[idx_nonzer, :],
    #edge_threshold="20%", # 0.9 or "15%"
    #title="Avg importance of overlapping features",
    title=None,
    node_size=15,
    black_bg=False,
    alpha=0.5,
    output_file='figures/brain_plots/avg_importance.png',
    node_color='black'
    
);

In [32]:
from IPython.display import Image
Image("figures/avg_importance.png")

In [33]:
sns.heatmap(data=symmetric_matrix[idx_nonzer, :][:, idx_nonzer], vmin=0.0, vmax=0.12,
           xticklabels=idx_nonzer, yticklabels=idx_nonzer);

### Little Brains for AEC, ATM, PLV feature important edges (top 20)

In [34]:
xgb_imp_aec = pd.read_csv("data/feature_importance/aec_xgb_importance.csv", index_col=0)
xgb_imp_atm = pd.read_csv("data/feature_importance/atm_xgb_importance.csv", index_col=0)
xgb_imp_plv = pd.read_csv("data/feature_importance/plv_xgb_importance.csv", index_col=0)

In [35]:
tmp = [num.split('-') for num in list(xgb_imp_aec.index)]
xgb_imp_aec.index = [''.join([str(int(part) - 1) for part in num]) for num in tmp]

tmp = [num.split('-') for num in list(xgb_imp_atm.index)]
xgb_imp_atm.index = [''.join([str(int(part) - 1) for part in num]) for num in tmp]

tmp = [num.split('-') for num in list(xgb_imp_plv.index)]
xgb_imp_plv.index = [''.join([str(int(part) - 1) for part in num]) for num in tmp]

In [36]:
for k in [xgb_imp_aec, xgb_imp_atm, xgb_imp_plv]:
    matr_zer = np.zeros((116, 116))

    for index, value in k.iterrows():
        if len(index) <= 4:
            row, col = divmod(int(index), 100)
        elif len(index) == 5:
            row, col = divmod(int(index), 1000)
        #print(f'row: {row}, id: {col}')
        matr_zer[row, col] = value.values[0]
        
    # regions of AAL from library and our - are in different order so we should reorder them
    matr_zer = matr_zer[indices_aal, :][:, indices_aal]
        
    idx_coords = np.unique(np.array(np.nonzero(matr_zer)).flatten())
    symmetric_matrix = matr_zer + matr_zer.T # make a symmetric matrix to have undirected links
    
    if k is xgb_imp_aec:
        file_name = 'xgb_imp_aec'
    elif k is xgb_imp_atm:
        file_name = 'xgb_imp_atm'
    elif k is xgb_imp_plv:
        file_name = 'xgb_imp_plv'
    plotting.plot_connectome(
        symmetric_matrix[idx_coords,:][:,idx_coords],
        coordinates[idx_coords, :],
        title=None,
        node_size=15,
        black_bg=False,
        alpha=0.5,
        output_file=f'figures/brain_plots/{file_name}.png',
        node_color='black'
    );